In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io 
import scipy.misc
import matplotlib.cm as cm
import random
from scipy.special import expit

In [17]:
datafile = 'C:\\Users\\Dell\\OneDrive\\Desktop\\DSA\\ML\\CS229_assignments\\EX3\\ex3data1.mat'
mat = scipy.io.loadmat( datafile )
X, y = mat['X'], mat['y']
X = np.insert(X,0,1,axis=1)
print("'y' shape: %s. Unique elements in y: %s"%(mat['y'].shape,np.unique(mat['y'])))
print("'X' shape: %s. X[0] shape: %s"%(X.shape,X[0].shape))

'y' shape: (5000, 1). Unique elements in y: [ 1  2  3  4  5  6  7  8  9 10]
'X' shape: (5000, 401). X[0] shape: (401,)


In [18]:
X, y

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8))

In [21]:
def h(mytheta,myX):
    return expit(np.dot(myX,mytheta))

In [22]:
def computeCost(mytheta,myX,myy,mylambda = 0.):
    m = myX.shape[0] #5000
    myh = h(mytheta,myX) #shape: (5000,1)
    term1 = np.log( myh ).dot( -myy.T ) #shape: (5000,5000)
    term2 = np.log( 1.0 - myh ).dot( 1 - myy.T ) #shape: (5000,5000)
    left_hand = (term1 - term2) / m #shape: (5000,5000)
    right_hand = mytheta.T.dot( mytheta ) * mylambda / (2*m) #shape: (1,1)
    return left_hand + right_hand #shape: (5000,5000)

In [23]:
def costGradient(mytheta,myX,myy,mylambda = 0.):
    m = myX.shape[0]
    beta = h(mytheta,myX)-myy.T

    regterm = mytheta[1:]*(mylambda/m) #shape: (400,1)
    grad = (1./m)*np.dot(myX.T,beta) #shape: (401, 5000)
    grad[1:] = grad[1:] + regterm
    return grad #shape: (401, 5000)

In [24]:
from scipy import optimize
def optimizeTheta(mytheta,myX,myy,mylambda=0.):
    result = optimize.fmin_cg(computeCost, fprime=costGradient, x0=mytheta, \
                              args=(myX, myy, mylambda), maxiter=50, disp=False,\
                              full_output=True)
    return result[0], result[1]

In [25]:
def buildTheta():
    mylambda = 0.
    initial_theta = np.zeros((X.shape[1],1)).reshape(-1)
    Theta = np.zeros((10,X.shape[1]))
    for i in range(10):
        iclass = i if i else 10 
        print("Optimizing for handwritten number %d..."%i)
        # reshape((X.shape[0],1))
        logic_Y = np.array([1 if x == iclass else 0 for x in y])
        itheta, imincost = optimizeTheta(initial_theta,X,logic_Y,mylambda)
        Theta[i,:] = itheta
    print("Done!")
    return Theta

In [26]:
Theta = buildTheta()

Optimizing for handwritten number 0...
Optimizing for handwritten number 1...
Optimizing for handwritten number 2...
Optimizing for handwritten number 3...
Optimizing for handwritten number 4...
Optimizing for handwritten number 5...
Optimizing for handwritten number 6...
Optimizing for handwritten number 7...
Optimizing for handwritten number 8...
Optimizing for handwritten number 9...
Done!


In [27]:
print(Theta)

[[-8.01221263e+00  0.00000000e+00  0.00000000e+00 ... -4.57543305e-04
   2.23253475e-05  0.00000000e+00]
 [-3.05422655e+00  0.00000000e+00  0.00000000e+00 ...  4.11428941e-03
   2.59650608e-07  0.00000000e+00]
 [-2.86285942e+00  0.00000000e+00  0.00000000e+00 ...  5.87023922e-03
  -6.70822503e-04  0.00000000e+00]
 ...
 [-2.74038255e+00  0.00000000e+00  0.00000000e+00 ... -1.53592172e-03
   1.90101331e-04  0.00000000e+00]
 [-8.69371256e+00  0.00000000e+00  0.00000000e+00 ... -1.22687182e-04
   1.17820239e-05  0.00000000e+00]
 [-5.50736739e+00  0.00000000e+00  0.00000000e+00 ... -1.21616616e-03
   9.25597799e-05  0.00000000e+00]]


In [28]:
Theta.shape

(10, 401)

In [29]:
def predictOneVsAll(myTheta,myrow):
    classes = [10] + range(1,10)
    hypots  = [0]*len(classes)
    for i in xrange(len(classes)):
        hypots[i] = h(myTheta[i],myrow)
    return classes[np.argmax(np.array(hypots))]